In [1]:
#!usr/bin/env python3
import os
import pandas as pd
import dateutil #https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
from datetime import datetime
import glob
import gc  #garbage collection to free up memory

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')


In [2]:
#set some static parameters
debug_mode = 'n'
csv_header_ind = 'True'
cur_dir = os.getcwd()

## Load Data

In [3]:
if debug_mode == 'y':
    data_year = 9999
    print(data_year)
else:
    for i in range(2013,2015):
        data_year = i
        print(data_year)        

2013
2014


In [4]:
        # print start timestamp 
        execStartDateTime = datetime.now()
        print(execStartDateTime)

2020-06-22 05:52:15.352331


In [5]:
        raw_dir = os.path.join(cur_dir,'citibike_files','raw', str(data_year))

In [6]:
        path = raw_dir

        all_files = sorted(glob.glob(os.path.join(path, "*citibike*")))

        all_df = []

        #parse_dates=['starttime','stoptime'],

        for f in all_files:
            try: 
                print('try:' + f)
                df = pd.read_csv(f, sep=',',header = 1,skiprows = 1,\
                names=['tripduration','starttime','stoptime','start station id','start station name'\
                ,'start station latitude','start station longitude','end station id','end station name'\
                ,'end station latitude','end station longitude','bikeid','usertype','birth year','gender'])
                print(len(df.index))
                df['a_file'] = f.split('/')[-1]    
                all_df.append(df)
                citibike_df = pd.concat(all_df, ignore_index=True, sort=True)
            except:
                 print('except:' + f)

try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201401-citibike-tripdata.zip
300398
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201402-citibike-tripdata.zip
224734
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201403-citibike-tripdata.zip
439115
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201404-citibike-tripdata.zip
670778
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201405-citibike-tripdata.zip
866115
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201406-citibike-tripdata.zip
936878
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201407-citibike-tripdata.zip
968840
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201408-citibike-tripdata.zip
963487
try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/2014/201409-citibike-tripdata.zip
953885
try:/Users

## Pre-processing: Preview data & datatype inspection

In [7]:
        citibike_df= pd.DataFrame(citibike_df)

In [8]:
        list(citibike_df.columns) 

['a_file',
 'bikeid',
 'birth year',
 'end station id',
 'end station latitude',
 'end station longitude',
 'end station name',
 'gender',
 'start station id',
 'start station latitude',
 'start station longitude',
 'start station name',
 'starttime',
 'stoptime',
 'tripduration',
 'usertype']

In [9]:
        citibike_df.dtypes

a_file                      object
bikeid                       int64
birth year                  object
end station id               int64
end station latitude       float64
end station longitude      float64
end station name            object
gender                       int64
start station id             int64
start station latitude     float64
start station longitude    float64
start station name          object
starttime                   object
stoptime                    object
tripduration                 int64
usertype                    object
dtype: object

In [10]:
        # set text columns as categories
        for col in ['gender', 'usertype', 'start station name', 'end station name']:
            citibike_df[col] = citibike_df[col].astype('category')

In [11]:
        # set datatypes for numeric columns
        citibike_df['start station id'] = citibike_df['start station id'].astype(str).astype(float).astype(int)
        citibike_df['start station latitude'] = citibike_df['start station latitude'].astype(float)
        citibike_df['start station latitude'] = citibike_df['start station latitude'].round(decimals=3)
        citibike_df['start station longitude'] = citibike_df['start station longitude'].astype(float)
        citibike_df['start station longitude'] = citibike_df['start station longitude'].round(decimals=3)
        citibike_df = citibike_df.dropna(subset=['end station id'])
        citibike_df['end station id'] = citibike_df['end station id'].astype(str).astype(float).astype(int)
        citibike_df['end station latitude'] = citibike_df['end station latitude'].astype(float)
        citibike_df['end station latitude'] = citibike_df['end station latitude'].round(decimals=3)
        citibike_df['end station longitude'] = citibike_df['end station longitude'].astype(float)
        citibike_df['end station longitude'] = citibike_df['end station longitude'].round(decimals=3)

In [12]:
        citibike_df.dtypes

a_file                       object
bikeid                        int64
birth year                   object
end station id                int64
end station latitude        float64
end station longitude       float64
end station name           category
gender                     category
start station id              int64
start station latitude      float64
start station longitude     float64
start station name         category
starttime                    object
stoptime                     object
tripduration                  int64
usertype                   category
dtype: object

In [13]:
        citibike_df.head()

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
0,201401-citibike-tripdata.zip,16613,1991,2022,40.758,-73.959,E 59 St & Sutton Pl,1,228,40.755,-73.972,E 48 St & 3 Ave,2014-01-01 00:03:59,2014-01-01 00:11:43,464,Subscriber
1,201401-citibike-tripdata.zip,15938,1989,526,40.748,-73.985,E 33 St & 5 Ave,1,519,40.752,-73.978,Pershing Square N,2014-01-01 00:05:15,2014-01-01 00:11:28,373,Subscriber
2,201401-citibike-tripdata.zip,19830,1990,436,40.682,-73.954,Hancock St & Bedford Ave,1,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl,2014-01-01 00:05:18,2014-01-01 00:16:18,660,Subscriber
3,201401-citibike-tripdata.zip,17343,1987,526,40.748,-73.985,E 33 St & 5 Ave,1,422,40.771,-73.988,W 59 St & 10 Ave,2014-01-01 00:05:55,2014-01-01 00:11:25,330,Subscriber
4,201401-citibike-tripdata.zip,17880,1983,167,40.749,-73.976,E 39 St & 3 Ave,1,516,40.752,-73.968,E 47 St & 1 Ave,2014-01-01 00:06:04,2014-01-01 00:10:25,261,Subscriber


In [14]:
        citibike_df['birth year'].value_counts()

\N        580777
1983      209164
1984      209085
1985      207730
1986      194392
           ...  
1931.0         8
1925.0         6
1929           5
1927.0         4
1913           4
Name: birth year, Length: 164, dtype: int64

In [15]:
        # Using try block here since data files were not consistent over time
        try:
            if pd.api.types.is_string_dtype:
                citibike_df['birth year'] = citibike_df['birth year'].replace({"\\N":2020})
        except:
            print("skip")

In [16]:
        citibike_df['birth year'].fillna(2020,inplace=True)

In [17]:
        # Now that all fields are prepped drop nans in dataframe.  This is slow.
        citibike_df.dropna(inplace=True)

In [19]:
        citibike_df.isnull().sum(axis=0)

a_file                     0
bikeid                     0
birth year                 0
end station id             0
end station latitude       0
end station longitude      0
end station name           0
gender                     0
start station id           0
start station latitude     0
start station longitude    0
start station name         0
starttime                  0
stoptime                   0
tripduration               0
usertype                   0
dtype: int64

In [ ]:
        # Convert trip duration from seconds to minutes
        citibike_df['tripduration'] = citibike_df['tripduration']/60

In [18]:
        # Set birth year datatype the nans dropped
        citibike_df['birth year'] = citibike_df['birth year'].astype(str).astype(float).astype(int)

In [20]:
        # Stamp the output files yearmonth to track the source of the data
        citibike_df['yearmonth'] =  citibike_df['a_file'].str[:6].astype(int)

## Analyze by date and starthour

In [21]:
        citibike_df.dtypes

a_file                       object
bikeid                        int64
birth year                    int64
end station id                int64
end station latitude        float64
end station longitude       float64
end station name           category
gender                     category
start station id              int64
start station latitude      float64
start station longitude     float64
start station name         category
starttime                    object
stoptime                     object
tripduration                float64
usertype                   category
yearmonth                     int64
dtype: object

In [22]:
        citibike_df[['begindate','begintime']] = citibike_df.starttime.str.split(expand=True) 

In [23]:
        # Possible optimzation:  https://stackoverflow.com/questions/50744369/how-to-speed-up-pandas-string-function
        # %timeit [x.split('~', 1)[0] for x in df['facility']]
        # def splittime(x):
        #     test = [x.split(' ', 1)[0] for x in citibike_df['starttime']]
        #     return x.map(test)
        # citibike_df['test2'] = splittime(citibike_df['starttime'])
        # TypeError: list indices must be integers or slices, not str   

In [24]:
        # https://github.com/pandas-dev/pandas/issues/11665
        def lookup(s):
            """
            This is an extremely fast approach to datetime parsing.
            For large data, the same dates are often repeated. Rather than
            re-parse these, we store all unique dates, parse them, and
            use a lookup to convert all dates.
            """
            dates = {date:pd.to_datetime(date) for date in s.unique()}
            return s.map(dates)

In [25]:
        citibike_df['startdate'] = lookup(citibike_df['begindate'])

In [26]:
        citibike_df.head()

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype,yearmonth,begindate,begintime,startdate
0,201401-citibike-tripdata.zip,16613,1991,2022,40.758,-73.959,E 59 St & Sutton Pl,1,228,40.755,-73.972,E 48 St & 3 Ave,2014-01-01 00:03:59,2014-01-01 00:11:43,7.733333,Subscriber,201401,2014-01-01,00:03:59,2014-01-01
1,201401-citibike-tripdata.zip,15938,1989,526,40.748,-73.985,E 33 St & 5 Ave,1,519,40.752,-73.978,Pershing Square N,2014-01-01 00:05:15,2014-01-01 00:11:28,6.216667,Subscriber,201401,2014-01-01,00:05:15,2014-01-01
2,201401-citibike-tripdata.zip,19830,1990,436,40.682,-73.954,Hancock St & Bedford Ave,1,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl,2014-01-01 00:05:18,2014-01-01 00:16:18,11.000000,Subscriber,201401,2014-01-01,00:05:18,2014-01-01
3,201401-citibike-tripdata.zip,17343,1987,526,40.748,-73.985,E 33 St & 5 Ave,1,422,40.771,-73.988,W 59 St & 10 Ave,2014-01-01 00:05:55,2014-01-01 00:11:25,5.500000,Subscriber,201401,2014-01-01,00:05:55,2014-01-01
4,201401-citibike-tripdata.zip,17880,1983,167,40.749,-73.976,E 39 St & 3 Ave,1,516,40.752,-73.968,E 47 St & 1 Ave,2014-01-01 00:06:04,2014-01-01 00:10:25,4.350000,Subscriber,201401,2014-01-01,00:06:04,2014-01-01


In [27]:
        citibike_df['starthour'] = citibike_df['begintime'].str.slice(0, 2)

In [28]:
        daily_df = citibike_df.groupby(['startdate']).tripduration.agg(['count','sum']).reset_index().set_index(['startdate'])
        daily_df.sort_index(axis = 0) 
        daily_df

,count,sum
startdate,,
2014-01-01,6057,93139.183333
2014-01-02,8600,111630.816667
2014-01-03,1144,15316.866667
2014-01-04,2292,46752.783333
2014-01-05,2678,36788.500000
...,...,...
2014-12-27,10247,165377.800000
2014-12-28,7902,103078.233333
2014-12-29,13549,179936.916667


In [29]:
        citibike_daily_bike_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_trips_daily.csv')

In [30]:
        if debug_mode == 'n':
            if not os.path.isfile(citibike_daily_bike_csv):
               daily_df.to_csv(citibike_daily_bike_csv, header='column_names', index=False)
            else: # else it exists so append without writing the header
               daily_df.to_csv(citibike_daily_bike_csv, mode='a', header=False, index=False)

In [31]:
        # Extend analysis tostart hour
        hourly_df = citibike_df.groupby(['startdate','starthour']).tripduration.agg(['count','sum']).reset_index()
        hourly_df.set_index('startdate', inplace = True) 
        hourly_df.sort_index(axis = 0) 
        hourly_df.head()

,starthour,count,sum
startdate,,,
2014-01-01,00,113,1737.100000
2014-01-01,01,204,4577.666667
2014-01-01,02,161,4045.116667
2014-01-01,03,144,3795.016667
2014-01-01,04,61,521.366667


In [32]:
        citibike_hourly_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_trips_hourly.csv')

In [33]:
        if debug_mode == 'n':
            if not os.path.isfile(citibike_hourly_csv):
               hourly_df.to_csv(citibike_hourly_csv, header='column_names', index=False)
            else: # else it exists so append without writing the header
               hourly_df.to_csv(citibike_hourly_csv, mode='a', header=False, index=False)

## Analyze customer data

In [34]:
        citibike_df['gender'].value_counts()

1    5635944
2    1650270
0     794978
Name: gender, dtype: int64

In [35]:
        citibike_df['birth year'].value_counts()

2020    793710
1983    312520
1984    310087
1985    308572
1986    296037
         ...  
1927        17
1920         8
1925         6
1929         5
1913         4
Name: birth year, Length: 85, dtype: int64

In [36]:
        currentYear = datetime.now().year

In [37]:
        citibike_df['rider age'] = currentYear - citibike_df['birth year']
        citibike_df

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,...,starttime,stoptime,tripduration,usertype,yearmonth,begindate,begintime,startdate,starthour,rider age
0,201401-citibike-tripdata.zip,16613,1991,2022,40.758,-73.959,E 59 St & Sutton Pl,1,228,40.755,...,2014-01-01 00:03:59,2014-01-01 00:11:43,7.733333,Subscriber,201401,2014-01-01,00:03:59,2014-01-01,00,29
1,201401-citibike-tripdata.zip,15938,1989,526,40.748,-73.985,E 33 St & 5 Ave,1,519,40.752,...,2014-01-01 00:05:15,2014-01-01 00:11:28,6.216667,Subscriber,201401,2014-01-01,00:05:15,2014-01-01,00,31
2,201401-citibike-tripdata.zip,19830,1990,436,40.682,-73.954,Hancock St & Bedford Ave,1,83,40.684,...,2014-01-01 00:05:18,2014-01-01 00:16:18,11.000000,Subscriber,201401,2014-01-01,00:05:18,2014-01-01,00,30
3,201401-citibike-tripdata.zip,17343,1987,526,40.748,-73.985,E 33 St & 5 Ave,1,422,40.771,...,2014-01-01 00:05:55,2014-01-01 00:11:25,5.500000,Subscriber,201401,2014-01-01,00:05:55,2014-01-01,00,33
4,201401-citibike-tripdata.zip,17880,1983,167,40.749,-73.976,E 39 St & 3 Ave,1,516,40.752,...,2014-01-01 00:06:04,2014-01-01 00:10:25,4.350000,Subscriber,201401,2014-01-01,00:06:04,2014-01-01,00,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8081187,201412-citibike-tripdata.zip,16908,1982,2008,40.706,-74.017,Little West St & 1 Pl,1,82,40.711,...,12/31/2014 23:55:22,1/1/2015 00:04:35,9.216667,Subscriber,201412,12/31/2014,23:55:22,2014-12-31,23,38
8081188,201412-citibike-tripdata.zip,20964,1971,327,40.715,-74.017,Vesey Pl & River Terrace,1,325,40.736,...,12/31/2014 23:56:37,1/1/2015 00:19:12,22.583333,Subscriber,201412,12/31/2014,23:56:37,2014-12-31,23,49
8081189,201412-citibike-tripdata.zip,20845,1963,491,40.741,-73.986,E 24 St & Park Ave S,1,536,40.741,...,12/31/2014 23:57:03,1/1/2015 00:04:14,7.183333,Subscriber,201412,12/31/2014,23:57:03,2014-12-31,23,57
8081190,201412-citibike-tripdata.zip,15374,1984,317,40.725,-73.982,E 6 St & Avenue B,1,293,40.730,...,12/31/2014 23:57:25,1/1/2015 00:02:57,5.533333,Subscriber,201412,12/31/2014,23:57:25,2014-12-31,23,36


In [38]:
        bins = [-1,1,18,25,45,65,100,1000]
        citibike_df['age bracket'] = pd.cut(citibike_df['rider age'],bins)
        citibike_df

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,...,stoptime,tripduration,usertype,yearmonth,begindate,begintime,startdate,starthour,rider age,age bracket
0,201401-citibike-tripdata.zip,16613,1991,2022,40.758,-73.959,E 59 St & Sutton Pl,1,228,40.755,...,2014-01-01 00:11:43,7.733333,Subscriber,201401,2014-01-01,00:03:59,2014-01-01,00,29,"(25, 45]"
1,201401-citibike-tripdata.zip,15938,1989,526,40.748,-73.985,E 33 St & 5 Ave,1,519,40.752,...,2014-01-01 00:11:28,6.216667,Subscriber,201401,2014-01-01,00:05:15,2014-01-01,00,31,"(25, 45]"
2,201401-citibike-tripdata.zip,19830,1990,436,40.682,-73.954,Hancock St & Bedford Ave,1,83,40.684,...,2014-01-01 00:16:18,11.000000,Subscriber,201401,2014-01-01,00:05:18,2014-01-01,00,30,"(25, 45]"
3,201401-citibike-tripdata.zip,17343,1987,526,40.748,-73.985,E 33 St & 5 Ave,1,422,40.771,...,2014-01-01 00:11:25,5.500000,Subscriber,201401,2014-01-01,00:05:55,2014-01-01,00,33,"(25, 45]"
4,201401-citibike-tripdata.zip,17880,1983,167,40.749,-73.976,E 39 St & 3 Ave,1,516,40.752,...,2014-01-01 00:10:25,4.350000,Subscriber,201401,2014-01-01,00:06:04,2014-01-01,00,37,"(25, 45]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8081187,201412-citibike-tripdata.zip,16908,1982,2008,40.706,-74.017,Little West St & 1 Pl,1,82,40.711,...,1/1/2015 00:04:35,9.216667,Subscriber,201412,12/31/2014,23:55:22,2014-12-31,23,38,"(25, 45]"
8081188,201412-citibike-tripdata.zip,20964,1971,327,40.715,-74.017,Vesey Pl & River Terrace,1,325,40.736,...,1/1/2015 00:19:12,22.583333,Subscriber,201412,12/31/2014,23:56:37,2014-12-31,23,49,"(45, 65]"
8081189,201412-citibike-tripdata.zip,20845,1963,491,40.741,-73.986,E 24 St & Park Ave S,1,536,40.741,...,1/1/2015 00:04:14,7.183333,Subscriber,201412,12/31/2014,23:57:03,2014-12-31,23,57,"(45, 65]"
8081190,201412-citibike-tripdata.zip,15374,1984,317,40.725,-73.982,E 6 St & Avenue B,1,293,40.730,...,1/1/2015 00:02:57,5.533333,Subscriber,201412,12/31/2014,23:57:25,2014-12-31,23,36,"(25, 45]"


In [39]:
        customer_df = citibike_df.groupby(['startdate','gender','age bracket','usertype']).tripduration.agg(['count']).reset_index()
        customer_df

,startdate,gender,age bracket,usertype,count
0,2014-01-01,0,"(-1, 1]",Customer,652
1,2014-01-01,0,"(-1, 1]",Subscriber,0
2,2014-01-01,0,"(1, 18]",Customer,0
3,2014-01-01,0,"(1, 18]",Subscriber,0
4,2014-01-01,0,"(18, 25]",Customer,0
...,...,...,...,...,...
15325,2014-12-31,2,"(45, 65]",Subscriber,638
15326,2014-12-31,2,"(65, 100]",Customer,0
15327,2014-12-31,2,"(65, 100]",Subscriber,79
15328,2014-12-31,2,"(100, 1000]",Customer,0


In [40]:
        citibike_customer_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_customers.csv')

In [41]:
        if debug_mode == 'n':
            #https://stackoverflow.com/questions/30991541/pandas-write-csv-append-vs-write
            if not os.path.isfile(citibike_customer_csv):
               customer_df.to_csv(citibike_customer_csv, header='column_names', index=False)
            else: # else it exists so append without writing the header
               customer_df.to_csv(citibike_customer_csv, mode='a', header=False, index=False)


## Analyze bike stations

In [42]:
        start_stations_df = citibike_df.drop_duplicates(subset=["start station id", "start station latitude","start station longitude","start station name"])
        start_stations_df = start_stations_df[["start station id", "start station latitude","start station longitude","start station name"]]
        start_stations_df = pd.DataFrame(start_stations_df)
        start_stations_df.columns = ["station id", "station latitude","station longitude","station name"]
        start_stations_df

,station id,station latitude,station longitude,station name
0,228,40.755,-73.972,E 48 St & 3 Ave
1,519,40.752,-73.978,Pershing Square N
2,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
3,422,40.771,-73.988,W 59 St & 10 Ave
4,516,40.752,-73.968,E 47 St & 1 Ave
...,...,...,...,...
2501815,279,40.708,-74.002,Peck Slip & Front Street
5370584,519,40.752,-73.978,Pershing Square North
5372031,517,40.752,-73.978,Pershing Square South
7390775,538,40.757,-73.977,E 48 St & Madison Ave


In [43]:
        end_stations_df = citibike_df.drop_duplicates(subset=["end station id", "end station latitude","end station longitude","end station name"])
        end_stations_df = end_stations_df[["end station id", "end station latitude","end station longitude","end station name"]]
        end_stations_df = pd.DataFrame(end_stations_df)
        end_stations_df.columns = ["station id", "station latitude","station longitude","station name"]
        end_stations_df

,station id,station latitude,station longitude,station name
0,2022,40.758,-73.959,E 59 St & Sutton Pl
1,526,40.748,-73.985,E 33 St & 5 Ave
2,436,40.682,-73.954,Hancock St & Bedford Ave
4,167,40.749,-73.976,E 39 St & 3 Ave
5,435,40.742,-73.994,W 21 St & 6 Ave
...,...,...,...,...
2502078,243,40.688,-73.979,Fulton St & Rockwell Pl
5370366,519,40.752,-73.978,Pershing Square North
5371172,517,40.752,-73.978,Pershing Square South
7388946,538,40.757,-73.977,E 48 St & Madison Ave


In [44]:
        distinct_stations_df = pd.DataFrame(start_stations_df.append(end_stations_df))
        distinct_stations_df

,station id,station latitude,station longitude,station name
0,228,40.755,-73.972,E 48 St & 3 Ave
1,519,40.752,-73.978,Pershing Square N
2,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
3,422,40.771,-73.988,W 59 St & 10 Ave
4,516,40.752,-73.968,E 47 St & 1 Ave
...,...,...,...,...
2502078,243,40.688,-73.979,Fulton St & Rockwell Pl
5370366,519,40.752,-73.978,Pershing Square North
5371172,517,40.752,-73.978,Pershing Square South
7388946,538,40.757,-73.977,E 48 St & Madison Ave


In [45]:
        distinct_stations_df = distinct_stations_df.drop_duplicates(subset=["station id", "station latitude","station longitude","station name"])
        distinct_stations_df

,station id,station latitude,station longitude,station name
0,228,40.755,-73.972,E 48 St & 3 Ave
1,519,40.752,-73.978,Pershing Square N
2,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
3,422,40.771,-73.988,W 59 St & 10 Ave
4,516,40.752,-73.968,E 47 St & 1 Ave
...,...,...,...,...
2501815,279,40.708,-74.002,Peck Slip & Front Street
5370584,519,40.752,-73.978,Pershing Square North
5372031,517,40.752,-73.978,Pershing Square South
7390775,538,40.757,-73.977,E 48 St & Madison Ave


In [46]:
        distinct_stations_df.set_index('station id', inplace = True)
        distinct_stations_df.head()

,station latitude,station longitude,station name
station id,,,
228,40.755,-73.972,E 48 St & 3 Ave
519,40.752,-73.978,Pershing Square N
83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
422,40.771,-73.988,W 59 St & 10 Ave
516,40.752,-73.968,E 47 St & 1 Ave


In [47]:
        distinct_stations_df.sort_index(axis = 0) 
        distinct_stations_df

,station latitude,station longitude,station name
station id,,,
228,40.755,-73.972,E 48 St & 3 Ave
519,40.752,-73.978,Pershing Square N
83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
422,40.771,-73.988,W 59 St & 10 Ave
516,40.752,-73.968,E 47 St & 1 Ave
...,...,...,...
279,40.708,-74.002,Peck Slip & Front Street
519,40.752,-73.978,Pershing Square North
517,40.752,-73.978,Pershing Square South


In [48]:
        distinct_stations_df.isnull().sum()

station latitude     0
station longitude    0
station name         0
dtype: int64

In [49]:
        citibike_distinct_station_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_distinct_stations.csv')

In [50]:
        if debug_mode == 'n':
            if not os.path.isfile(citibike_distinct_station_csv):
               distinct_stations_df.to_csv(citibike_distinct_station_csv, header='column_names')
            else: # else it exists so append without writing the header
               distinct_stations_df.to_csv(citibike_distinct_station_csv, mode='a', header=False)

In [51]:
        start_station_trips_df = citibike_df.groupby(['startdate','start station id', 'start station latitude', 'start station longitude']).tripduration.agg(['count']).reset_index()
        start_station_trips_df = start_station_trips_df.set_index(['startdate'])
        start_station_trips_df

,start station id,start station latitude,start station longitude,count
startdate,,,,
2014-01-01,72,40.767,-73.994,6
2014-01-01,79,40.719,-74.007,17
2014-01-01,82,40.711,-74.000,13
2014-01-01,83,40.684,-73.976,17
2014-01-01,116,40.742,-74.001,28
...,...,...,...,...
2014-12-31,2017,40.750,-73.971,18
2014-12-31,2021,40.759,-73.989,13
2014-12-31,2022,40.758,-73.959,20


In [52]:
        start_station_trips_df.isnull().sum()

start station id           0
start station latitude     0
start station longitude    0
count                      0
dtype: int64

In [53]:
        citibike_start_station_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_start_stations.csv')

In [54]:
        if debug_mode == 'n':
            if not os.path.isfile(citibike_start_station_csv):
               start_station_trips_df.to_csv(citibike_start_station_csv, header='column_names', index=False)
            else: # else it exists so append without writing the header
               start_station_trips_df.to_csv(citibike_start_station_csv, mode='a', header=False, index=False)

## Analyze bike equipment

In [55]:
        bike_equipment_df = citibike_df.groupby(['bikeid']).tripduration.agg(['count','sum']).reset_index()
        bike_equipment_df = bike_equipment_df.set_index('bikeid')
        bike_equipment_df = pd.DataFrame(bike_equipment_df)

In [56]:
        bike_date_df = citibike_df.groupby(['bikeid']).startdate.agg(['min','max']).reset_index()
        bike_date_df = bike_date_df.set_index(['bikeid'])
        bike_date_df = pd.DataFrame(bike_date_df)

In [57]:
        bike_merged_df = pd.merge(bike_date_df, bike_equipment_df, left_index=True, right_index=True)
        bike_merged_df = pd.DataFrame(bike_merged_df)

In [58]:
        citibike_bike_equipment_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_bikes.csv')

In [59]:
        if debug_mode == 'n':
            if not os.path.isfile(citibike_bike_equipment_csv):
               bike_merged_df.to_csv(citibike_bike_equipment_csv, header='column_names', index=False)
            else: # else it exists so append without writing the header
               bike_merged_df.to_csv(citibike_bike_equipment_csv, mode='a', header=False)

## Cleanup memory for next run

In [60]:
        del [[citibike_df]]
        del [[start_stations_df, end_stations_df, distinct_stations_df]]
        del [[bike_equipment_df, bike_date_df, bike_merged_df]]
        gc.collect()
        citibike_df = []
        customer_df = []
        distinct_stations_df = []
        start_stations_df = []
        end_stations_df = []
        bike_equipment_df = []
        bike_date_df = []
        bike_merged_df = []

## Post-Processing

In [61]:
# Data is processed by year.  Final aggregation needed to produce distinct stations for all years.
if debug_mode == 'n':
    citibike_distinct_station_final_df = pd.read_csv(os.path.join(cur_dir, 'citibike_files', 'cleansed', 'citibike_distinct_stations.csv'))
    citibike_distinct_station_final_df = pd.DataFrame(citibike_distinct_station_final_df)
    citibike_distinct_station_final_df = citibike_distinct_station_final_df.drop_duplicates(subset=["station id", "station latitude","station longitude","station name"],keep='first')   
    citibike_distinct_station_final_df = citibike_distinct_station_final_df[["station id", "station latitude","station longitude","station name"]]    
    citibike_distinct_station_final_df.columns = ["station id", "station latitude","station longitude","station name"]    
    citibike_distinct_station_final_df.set_index("station id", inplace=True)    
    citibike_distinct_station_final_df.sort_values("station id", axis = 0, ascending = True, inplace = True, na_position ='first')   
    citibike_distinct_station_final_df   
    citibike_distinct_station_final_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_distinct_stations_final.csv')       
    
    if not os.path.isfile(citibike_distinct_station_final_csv):
       citibike_distinct_station_final_df.to_csv(citibike_distinct_station_final_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       citibike_distinct_station_final_df.to_csv(citibike_distinct_station_final_csv, mode='a', header=False, index=False)

In [62]:
citibike_distinct_station_final_df.head()

,station latitude,station longitude,station name
station id,,,
72,40.767,-73.994,W 52 St & 11 Ave
79,40.719,-74.007,Franklin St & W Broadway
82,40.711,-74.000,St James Pl & Pearl St
83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
116,40.742,-74.001,W 17 St & 8 Ave


In [63]:
# print end 
print(data_year)
execEndDateTime = datetime.now()
print(execStartDateTime)
print(execEndDateTime)

2014
2020-06-22 05:52:15.352331
2020-06-22 05:55:11.601647
